In [1]:
%matplotlib notebook
import numpy as np
import h5py
import matplotlib.pyplot
import scipy.optimize as opt
import scipy.integrate as integrate
import dedalus.public as de

ModuleNotFoundError: No module named 'dedalus'

In [2]:
df = h5py.File("snapshots_s1.h5","r")

In [40]:
r = df['scales/r/1.0']
theta = df['scales/theta/1.0']
z = df['scales/z/1.0']

nr = r.shape[0]
nz = z.shape[0]
ntheta = theta.shape[0]

In [30]:
def find_endpoints(grid):
    N = grid.shape[0]
    index = np.arange(N)
    native_grid = np.cos(np.pi*(index +0.5)/N)
    
    a = (r[1]-r[0])/(native_grid[-2]-native_grid[-1])
    b = ((r[1]+r[0]) - a*(native_grid[-2]+native_grid[-1]))/2
    
    left = -a + b
    right = a + b
    
    return left, right

In [33]:
def find_fourier_right(grid):
    delta = grid[1]-grid[0]
    
    return grid[-1] + delta

In [34]:
R1, R2 = find_endpoints(r)
Lz = find_fourier_right(z)

In [41]:
rr = de.Chebyshev('r',nr,interval=[R1,R2])
ttheta = de.Fourier('theta',ntheta,interval=[0,2*np.pi])
zz = de.Fourier('z',nz, interval=[0,Lz])

domain = de.Domain([zz,ttheta,rr], grid_dtype='float')

In [42]:
u1 = domain.new_field()
u2 = domain.new_field()

u1['g'] = df['tasks/u'][-2]
u2['g'] = df['tasks/u'][-1]

In [43]:
delta_t = df['scales/sim_time'][-1]- df['scales/sim_time'][-2]

In [ ]:
def wave_speed(c,data1=data1,data2=data2,deltat=deltat):
    u_interp = u2.interpolate('theta=')